In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

pd.set_option('display.max_rows', 200)

In [2]:
# for parsing a timestamp when loading the data
parser = lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S %z %Z')

In [3]:
# loading the data and checking structures
consum = pd.read_csv('consumption_data.csv', parse_dates=[0], date_parser=parser)
mapp = pd.read_csv('feed_station_mapping.csv')
consum.head()

,Stamp,Track,Artist,Feed_Id,Feed_Name
0,2019-06-07 18:38:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183
1,2019-06-08 09:04:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183
2,2019-06-08 19:51:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183
3,2019-06-09 20:52:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183
4,2019-06-10 11:06:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183


In [4]:
mapp.head()

,Feed_Id,SMP_Station
0,feed_id_01,smp_station_name_1027
1,feed_id_02,smp_station_name_94
2,feed_id_03,smp_station_name_500
3,feed_id_04,smp_station_name_346
4,feed_id_06,smp_station_name_732


In [5]:
consum.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 521974 entries, 0 to 521973
Data columns (total 5 columns):
 #   Column     Non-Null Count   Dtype              
---  ------     --------------   -----              
 0   Stamp      521974 non-null  datetime64[ns, UTC]
 1   Track      521974 non-null  object             
 2   Artist     521974 non-null  object             
 3   Feed_Id    521974 non-null  object             
 4   Feed_Name  521974 non-null  object             
dtypes: datetime64[ns, UTC](1), object(4)
memory usage: 19.9+ MB


In [6]:
mapp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1125 entries, 0 to 1124
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Feed_Id      1125 non-null   object
 1   SMP_Station  1125 non-null   object
dtypes: object(2)
memory usage: 17.7+ KB


In [7]:
# consum dataset has more granular data, thus I want to keep that granularity and use left join
df = consum.merge(mapp, how='left', on='Feed_Id')

In [8]:
df.head()

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
0,2019-06-07 18:38:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183,smp_station_name_294
1,2019-06-08 09:04:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183,smp_station_name_294
2,2019-06-08 19:51:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183,smp_station_name_294
3,2019-06-09 20:52:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183,smp_station_name_294
4,2019-06-10 11:06:00+00:00,Song_02,artist_08,feed_id_1377,feed_name_1183,smp_station_name_294


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 521974 entries, 0 to 521973
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   Stamp        521974 non-null  datetime64[ns, UTC]
 1   Track        521974 non-null  object             
 2   Artist       521974 non-null  object             
 3   Feed_Id      521974 non-null  object             
 4   Feed_Name    521974 non-null  object             
 5   SMP_Station  486800 non-null  object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 27.9+ MB


In [10]:
# checking how many unique values each column has
cols = df.columns

for col in cols:
    print(col)
    print(df[col].nunique())

Stamp
384312
Track 
8
Artist
8
Feed_Id
1479
Feed_Name
1592
SMP_Station
1122


In [11]:
#checking where the duplications occur
df.groupby(['SMP_Station','Feed_Id']).size()

SMP_Station           Feed_Id     
smp_station_name_01   feed_id_68       37
smp_station_name_02   feed_id_342      71
smp_station_name_03   feed_id_23       31
smp_station_name_04   feed_id_1005    868
smp_station_name_05   feed_id_1028     70
                                     ... 
smp_station_name_995  feed_id_1171      7
smp_station_name_996  feed_id_139      65
smp_station_name_997  feed_id_591     131
smp_station_name_998  feed_id_946       8
smp_station_name_999  feed_id_106     102
Length: 1125, dtype: int64

In [12]:
# looking into further structure of those duplicates by looking at relevant station
df.loc[df['SMP_Station'] == 'smp_station_name_154'].sort_values(by='Stamp')

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
362195,2019-06-27 12:34:00+00:00,Song_05,artist_03,feed_id_736,feed_name_527,smp_station_name_154
362194,2019-06-28 08:40:00+00:00,Song_02,artist_08,feed_id_736,feed_name_527,smp_station_name_154
362200,2019-07-04 06:40:00+00:00,Song_02,artist_08,feed_id_736,feed_name_527,smp_station_name_154
362201,2019-07-16 07:38:00+00:00,Song_02,artist_08,feed_id_736,feed_name_527,smp_station_name_154
362203,2019-07-19 09:42:00+00:00,Song_04,artist_07,feed_id_736,feed_name_527,smp_station_name_154
362196,2019-07-24 08:41:00+00:00,Song_01,artist_02,feed_id_736,feed_name_527,smp_station_name_154
362204,2019-08-01 16:44:00+00:00,Song_04,artist_07,feed_id_736,feed_name_527,smp_station_name_154
362197,2019-08-17 12:26:00+00:00,Song_01,artist_02,feed_id_736,feed_name_527,smp_station_name_154
362198,2019-08-17 12:31:00+00:00,Song_01,artist_02,feed_id_736,feed_name_527,smp_station_name_154
362205,2019-08-20 09:49:00+00:00,Song_05,artist_03,feed_id_736,feed_name_527,smp_station_name_154


In [13]:
# it seems like the feed name is always the same for those different feeds within one station
# it will be a good proxy for those missing values in station name
st_x = df.loc[df['SMP_Station'] == 'smp_station_name_722'].sort_values(by='Stamp')
st_x.Feed_Name.nunique()

1

In [14]:
#removing the duplicate plays within one radio
cols = df.columns.tolist()
# as a timestamp is a the most important column to spot duplications, it can't be in the list to drop
cols.remove('Stamp') 

for col in cols:
    observed_cols = df.drop(col, axis=1).columns.tolist()
    df.drop_duplicates(observed_cols, keep='first', inplace=True)    

In [15]:
# in total 5523 records removed
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516451 entries, 0 to 521973
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   Stamp        516451 non-null  datetime64[ns, UTC]
 1   Track        516451 non-null  object             
 2   Artist       516451 non-null  object             
 3   Feed_Id      516451 non-null  object             
 4   Feed_Name    516451 non-null  object             
 5   SMP_Station  486719 non-null  object             
dtypes: datetime64[ns, UTC](1), object(5)
memory usage: 27.6+ MB


In [16]:
# double checking the duplications were dropped correctly
df.loc[df['SMP_Station'] == 'smp_station_name_154'].sort_values(by='Stamp')

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
362195,2019-06-27 12:34:00+00:00,Song_05,artist_03,feed_id_736,feed_name_527,smp_station_name_154
362194,2019-06-28 08:40:00+00:00,Song_02,artist_08,feed_id_736,feed_name_527,smp_station_name_154
362200,2019-07-04 06:40:00+00:00,Song_02,artist_08,feed_id_736,feed_name_527,smp_station_name_154
362201,2019-07-16 07:38:00+00:00,Song_02,artist_08,feed_id_736,feed_name_527,smp_station_name_154
362203,2019-07-19 09:42:00+00:00,Song_04,artist_07,feed_id_736,feed_name_527,smp_station_name_154
362196,2019-07-24 08:41:00+00:00,Song_01,artist_02,feed_id_736,feed_name_527,smp_station_name_154
362204,2019-08-01 16:44:00+00:00,Song_04,artist_07,feed_id_736,feed_name_527,smp_station_name_154
362197,2019-08-17 12:26:00+00:00,Song_01,artist_02,feed_id_736,feed_name_527,smp_station_name_154
362198,2019-08-17 12:31:00+00:00,Song_01,artist_02,feed_id_736,feed_name_527,smp_station_name_154
362205,2019-08-20 09:49:00+00:00,Song_05,artist_03,feed_id_736,feed_name_527,smp_station_name_154


In [17]:
# formatting the values before dealing with missing values

# removing characters from Track -> Feed_Id + SMP_Station values to keep only the numeric characters
cols = df.columns.tolist()
# no need to remove from timestamp, also need to 
cols = [c for c in cols if c not in ('Stamp','Feed_Name')]

for col in cols:
    df[col] = df[col].str.extract('(\d+)', expand=False)

# replacing 'feed_name_' to 'fn_' -> this way I will know which ones are
df['Feed_Name'] = df['Feed_Name'].replace('feed_name_', 'fn_', regex=True)

df.head()

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
0,2019-06-07 18:38:00+00:00,02,08,1377,fn_1183,294
1,2019-06-08 09:04:00+00:00,02,08,1377,fn_1183,294
2,2019-06-08 19:51:00+00:00,02,08,1377,fn_1183,294
3,2019-06-09 20:52:00+00:00,02,08,1377,fn_1183,294
4,2019-06-10 11:06:00+00:00,02,08,1377,fn_1183,294


In [18]:
# dealing with missing values
df[df.isnull().any(axis=1)].head(10)

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
387446,2019-01-16 03:44:25+00:00,01,02,569,fn_93,NaN
387447,2019-01-01 01:34:12+00:00,04,07,569,fn_93,NaN
387448,2019-01-09 06:27:42+00:00,04,07,569,fn_93,NaN
387449,2019-01-11 09:09:31+00:00,04,07,569,fn_93,NaN
387450,2019-01-16 00:40:36+00:00,04,07,569,fn_93,NaN
387451,2019-01-29 10:43:00+00:00,04,07,569,fn_93,NaN
387452,2019-02-02 06:47:35+00:00,04,07,569,fn_93,NaN
387453,2019-02-05 02:00:08+00:00,04,07,569,fn_93,NaN
387454,2019-02-13 04:55:46+00:00,04,07,569,fn_93,NaN
387455,2019-02-23 03:15:55+00:00,04,07,569,fn_93,NaN


In [19]:
# using Feed_Name as a station of missing values
df.SMP_Station.fillna(df.Feed_Name, inplace=True)

In [20]:
df.isna().sum()

Stamp          0
Track          0
Artist         0
Feed_Id        0
Feed_Name      0
SMP_Station    0
dtype: int64

In [21]:
# further cleaning:
# I will also reformat the time stamp to remove the timezone - as the dataset is of one region 
# Removing the fn_ from the Feed_Name column
# I also noticed that the column name is 'Track ', so I will remove the white space
df.head()

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
0,2019-06-07 18:38:00+00:00,02,08,1377,fn_1183,294
1,2019-06-08 09:04:00+00:00,02,08,1377,fn_1183,294
2,2019-06-08 19:51:00+00:00,02,08,1377,fn_1183,294
3,2019-06-09 20:52:00+00:00,02,08,1377,fn_1183,294
4,2019-06-10 11:06:00+00:00,02,08,1377,fn_1183,294


In [22]:
# formatting the timestamp
df['Stamp'] = df['Stamp'].dt.strftime('%Y-%m-%d %H:%M:%S')
df.head()

,Stamp,Track,Artist,Feed_Id,Feed_Name,SMP_Station
0,2019-06-07 18:38:00,02,08,1377,fn_1183,294
1,2019-06-08 09:04:00,02,08,1377,fn_1183,294
2,2019-06-08 19:51:00,02,08,1377,fn_1183,294
3,2019-06-09 20:52:00,02,08,1377,fn_1183,294
4,2019-06-10 11:06:00,02,08,1377,fn_1183,294


In [23]:
# removing the fn_
df['Feed_Name'] = df['Feed_Name'].str.extract('(\d+)', expand=False)

In [24]:
# removing whitespace from the column names
df.columns = df.columns.str.replace(' ','')
df.columns

Index(['Stamp', 'Track', 'Artist', 'Feed_Id', 'Feed_Name', 'SMP_Station'], dtype='object')

In [25]:
# exporting the dataset
df.to_csv('um_dataset.csv', index=False)